In [1]:
import sys
!{sys.executable} -m pip install opencv-python


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: /Users/shy/Projects/clippy/env/bin/python -m pip install --upgrade pip


In [ ]:
from pathlib import Path
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown
import cv2
import shutil
import ffmpeg
import subprocess
import os
from tqdm import tqdm

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

/Users/shy/Projects/clippy/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os

GOOGLE_API_KEY= os.environ['GOOGLE_API_KEY']
genai.configure(api_key=GOOGLE_API_KEY)

In [4]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)


models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [5]:
model = genai.GenerativeModel('gemini-pro')

In [9]:
response = model.generate_content("What is the meaning of life?")

In [10]:
to_markdown(response.text)

> **Meaning of Life**
> 
> The meaning of life is a philosophical question that has been pondered by humans for centuries. There is no definitive answer, but rather a multitude of perspectives and interpretations. Some common ideas include:
> 
> * **Purpose and Fulfillment:**
>     * Finding a purpose or mission that gives direction and motivation to one's life.
>     * Achieving personal goals, dreams, and aspirations.
> 
> * **Relationships and Connections:**
>     * Establishing meaningful relationships with family, friends, and loved ones.
>     * Contributing to society and making a positive impact on others.
> 
> * **Growth and Learning:**
>     * Pursuing knowledge, wisdom, and personal development.
>     * Continuously improving oneself and striving for higher ideals.
> 
> * **Ethics and Morals:**
>     * Living a life guided by ethical principles and values.
>     * Demonstrating compassion, empathy, and integrity.
> 
> * **Purpose in Nature:**
>     * Recognizing the interconnectedness of all life and the role humans play in the natural ecosystem.
>     * Appreciating the beauty and wonder of the natural world.
> 
> * **Personal Experiences:**
>     * Embracing and savoring life's experiences, both positive and negative.
>     * Recognizing the unique and subjective nature of meaning.
> 
> * **Spiritual Beliefs:**
>     * Connecting to a higher power or spiritual force.
>     * Seeking guidance, purpose, and meaning through religious or spiritual practices.
> 
> Ultimately, the meaning of life is a personal journey and discovery. It can be shaped by one's experiences, values, beliefs, and connections. While there may not be a universal answer, finding meaning in one's life can bring a sense of purpose, fulfillment, and peace.

In [ ]:
response.prompt_feedback

In [ ]:
response.candidates


In [ ]:
response = model.generate_content("What is the meaning of life?", stream=True)

## Pro-Vision Video!

In [11]:
model = genai.GenerativeModel('gemini-pro-vision')


In [12]:
!ls /Users/shy/Desktop/4-19-Sasha-blocked.MP4   

/Users/shy/Desktop/4-19-Sasha-blocked.MP4


In [26]:
FRAME_EXTRACTION_DIRECTORY = "frames"
FRAME_PREFIX = "_frame"

video_file_name = Path("/Users/shy/Desktop/4-19-Sasha-blocked.MP4")
video_file_name = Path("/Users/shy/Desktop/4-19-Sasha-dinner.MP4")
video_file_name = Path("/Users/shy/Desktop/test_clip.mp4")

audio_file_name = Path(f"{FRAME_EXTRACTION_DIRECTORY}/{video_file_name.stem}.mp3")

def call_ffmpeg(arg_string):
    subprocess.call(['ffmpeg', "-hide_banner", "-loglevel", "error", "-y"] + arg_string.split())
    
def extract_mp3_from_mp4(video_file_name, audio_file_name):
    call_ffmpeg(f"-i {video_file_name} {audio_file_name}")

extract_mp3_from_mp4(video_file_name, audio_file_name)

audio_file = genai.upload_file(path=audio_file_name)

In [27]:

# Create or cleanup existing extracted image frames directory.
def create_frame_output_dir(output_dir):
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)
  else:
    shutil.rmtree(output_dir)
    os.makedirs(output_dir)

def extract_frame_from_video(video_file_path):
  print(f"Extracting {video_file_path} at 1 frame per second. This might take a bit...")
  create_frame_output_dir(FRAME_EXTRACTION_DIRECTORY)
  vidcap = cv2.VideoCapture(str(video_file_path))
  fps = vidcap.get(cv2.CAP_PROP_FPS)
  frame_duration = 1 / fps  # Time interval between frames (in seconds)
  output_file_prefix = os.path.basename(video_file_path).replace('.', '_')
  frame_count = 0
  count = 0
  while vidcap.isOpened():
      success, frame = vidcap.read()
      if not success: # End of video
          break
      if int(count / fps) == frame_count: # Extract a frame every second
          min = frame_count // 60
          sec = frame_count % 60
          time_string = f"{min:02d}:{sec:02d}"
          image_name = f"{output_file_prefix}{FRAME_PREFIX}{time_string}.jpg"
          output_filename = os.path.join(FRAME_EXTRACTION_DIRECTORY, image_name)
          cv2.imwrite(output_filename, frame)
          frame_count += 1
      count += 1
  vidcap.release() # Release the capture object\n",
  print(f"Completed video frame extraction!\n\nExtracted: {frame_count} frames")

extract_frame_from_video(video_file_name)

Extracting /Users/shy/Desktop/test_clip.mp4 at 1 frame per second. This might take a bit...
Completed video frame extraction!

Extracted: 255 frames


In [29]:
class File:
  def __init__(self, file_path: str, display_name: str = None):
    self.file_path = file_path
    if display_name:
      self.display_name = display_name
    self.timestamp = get_timestamp(file_path)

  def set_file_response(self, response):
    self.response = response

def get_timestamp(filename):
  """Extracts the frame count (as an integer) from a filename with the format
     'output_file_prefix_frame00:00.jpg'.
  """
  parts = filename.split(FRAME_PREFIX)
  if len(parts) != 2:
      return None  # Indicates the filename might be incorrectly formatted
  return parts[1].split('.')[0]

# Process each frame in the output directory
files = os.listdir(FRAME_EXTRACTION_DIRECTORY)
files = sorted(files)
files_to_upload = []
for file in files:
  if not file.endswith('.jpg'):
    continue
  files_to_upload.append(
      File(file_path=os.path.join(FRAME_EXTRACTION_DIRECTORY, file)))

# Upload the files to the API
uploaded_files = []
print(f'Uploading {len(files_to_upload)} files. This might take a bit...')

for file in tqdm(files_to_upload):
  print(f'Uploading: {file.file_path}...')
  response = genai.upload_file(path=file.file_path)
  file.set_file_response(response)
  uploaded_files.append(file)

print(f"Completed file uploads!\n\nUploaded: {len(uploaded_files)} files")
     

Uploading 255 files. This might take a bit...
Uploading: frames/test_clip_mp4_frame00:00.jpg...
Uploading: frames/test_clip_mp4_frame00:01.jpg...
Uploading: frames/test_clip_mp4_frame00:02.jpg...
Uploading: frames/test_clip_mp4_frame00:03.jpg...
Uploading: frames/test_clip_mp4_frame00:04.jpg...
Uploading: frames/test_clip_mp4_frame00:05.jpg...
Uploading: frames/test_clip_mp4_frame00:06.jpg...
Uploading: frames/test_clip_mp4_frame00:07.jpg...
Uploading: frames/test_clip_mp4_frame00:08.jpg...
Uploading: frames/test_clip_mp4_frame00:09.jpg...
Uploading: frames/test_clip_mp4_frame00:10.jpg...
Uploading: frames/test_clip_mp4_frame00:11.jpg...
Uploading: frames/test_clip_mp4_frame00:12.jpg...
Uploading: frames/test_clip_mp4_frame00:13.jpg...
Uploading: frames/test_clip_mp4_frame00:14.jpg...
Uploading: frames/test_clip_mp4_frame00:15.jpg...
Uploading: frames/test_clip_mp4_frame00:16.jpg...
Uploading: frames/test_clip_mp4_frame00:17.jpg...
Uploading: frames/test_clip_mp4_frame00:18.jpg...
Uplo

In [ ]:
# List files uploaded in the API
for n, f in zip(range(len(uploaded_files)), genai.list_files()):
  print(f.uri)

https://generativelanguage.googleapis.com/v1beta/files/8kh3z8o62llp
https://generativelanguage.googleapis.com/v1beta/files/3zrba5l7rcpb
https://generativelanguage.googleapis.com/v1beta/files/zjb65hq5cxxw
https://generativelanguage.googleapis.com/v1beta/files/8hjfiv9sq49f
https://generativelanguage.googleapis.com/v1beta/files/8ixtlya0pk7h
https://generativelanguage.googleapis.com/v1beta/files/i733aa4u976a
https://generativelanguage.googleapis.com/v1beta/files/eoh4yk9re5h0
https://generativelanguage.googleapis.com/v1beta/files/xe1rcep4w9fc


In [ ]:
# Create the prompt.
prompt = "Describe this video."

# Set the model to Gemini 1.5 Pro.
model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest")

# Make GenerateContent request with the structure described above.
def make_request(prompt, files):
  request = [prompt]
  for file in files:
    request.append(file.timestamp)
    request.append(file.response)
  return request

# Make the LLM request.
request = make_request(prompt, uploaded_files) + [audio_file]
response = model.generate_content(request,
                                  request_options={"timeout": 600})
print(response.text)

The video starts with a blurry image of a light gray background. The audio contains two people talking about using the "magic of Gemini" to edit a video into a highlight reel.


In [ ]:
#print(f'Deleting {len(uploaded_files)} images. This might take a bit...')
#for file in uploaded_files:
#  genai.delete_file(file.response.name)
#  print(f'Deleted {file.file_path} at URI {file.response.uri}')
#print(f"Completed deleting files!\n\nDeleted: {len(uploaded_files)} files")